In [125]:
import os
import ast
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

RESULTS_PATH_1 = f"{os.curdir}/results/load-based-N-fields/tpch/2025-03-26-15H/meta_results.csv"

RESULTS_PATH_3 = f"{os.curdir}/results/load-based-N-fields/tpch/2025-05-28-21H/load9_results.csv"


In [126]:
# Define new color mapping for all tests (adjust as needed)
colors = {
    "schema_based_materialization": "#FF7F0E",  # Bright orange for clear visibility

    "no_materialization":  "#1f77b4",  # Blue
    "load_based_m1":  "#1f77b4",  # Green
    "load_based_m2":  "#1f77b4",  # Red
    "load_based_m3":  "#1f77b4",  # Purple
    "load_based_m4":  "#1f77b4",  # Brown

    "load_based_m5":  "#1f77b4",  # Blue
    "load_based_m6":  "#1f77b4",  # Green
    "load_based_m7":  "#1f77b4",  # Red
    "load_based_m8":  "#1f77b4",  # Purple
    "load_based_m9":  "#1f77b4",  # Brown

    "load_based_m10": "#1f77b4",  # Blue
    "load_based_m11": "#1f77b4",  # Green
    "load_based_m12": "#1f77b4",  # Red
    "load_based_m13": "#1f77b4",  # Purple
    "load_based_m14": "#1f77b4",  # Brown

    "load_based_m15": "#1f77b4",  # Blue
    "load_based_m20": "#1f77b4",  # Green
    "load_based_m25": "#1f77b4",  # Red
    "load_based_m30": "#1f77b4",  # Purple
    "load_based_m35": "#1f77b4",  # Brown
}


test_order = list(colors.keys())

def shorten_label_into_number(test):
    if test == "schema_based_materialization":
        return 20
    elif test == "no_materialization":
        return 0
    elif test.startswith("load_based_"):
        return int((test.split("_")[-1])[1:])
    else:
        return test
    

    # Helper function to shorten the labels
def shorten_label(test):
    if test == "schema_based_materialization":
        return "s16"
    elif test == "no_materialization":
        return "m0"
    elif test.startswith("load_based_"):
        return test.split("_")[-1]
    else:
        return test


In [127]:
def plot_load_results(df_load, load):
    """
    Create a bar plot showing total query time for different tests at a specific load.
    
    Args:
        df_load (pd.DataFrame): DataFrame containing results for a specific load
        load (int): Load value being plotted
    """
    # Create shortened labels for x-axis
    short_labels = [shorten_label(test) for test in df_load["Test"]]
    
    # Define positions and bar width
    x = np.arange(len(df_load))
    bar_width = 0.6

    # Create a new figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot bars using raw "Total query time" values
    ax.bar(
        x,
        df_load["Total Query Time"],
        bar_width,
        color=[colors[test] for test in df_load["Test"]]
    )

    
    # Set x-axis labels and titles
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels, rotation=45)
    ax.set_xlabel("Test")
    ax.set_ylabel("Total Query Time (s)")
    ax.set_title(f"Load {load}: Total Query Time")
    ax.grid(True, axis="y")
    plt.tight_layout()
    plt.show()


def plot_mean_results(mean_df):
    # Create shortened labels for x-axis
    short_labels = [shorten_label(test) for test in mean_df["Test"]]
    
    # Define positions and bar width
    x = np.arange(len(mean_df))
    bar_width = 0.6

    # Create a new figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot bars using raw "Total query time" values
    ax.bar(
        x,
        mean_df["mean"],
        bar_width,
        color=[colors[test] for test in mean_df["Test"]]
    )

    # Plot bars with error bars for standard error
    ax.bar(
        x,
        mean_df["mean"],
        bar_width,
        color=[colors[test] for test in mean_df["Test"]],
        yerr=mean_df["stderr"],
        capsize=5
    )
    
    # Set x-axis labels and titles
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels, rotation=45)
    ax.set_xlabel("Test")
    ax.set_ylabel("Total Query Time (s)")
    ax.set_title(f"Mean: Total Query Time")
    ax.grid(True, axis="y")
    plt.tight_layout()
    plt.show()

In [128]:
def prepare_df_for_plot(df):
    # Filter the DataFrame for the current load and tests in test_order
    df = df[(df["Test"].isin(test_order))].copy()
        
    # Ensure tests are ordered as defined in test_order
    df["Test"] = pd.Categorical(df["Test"], categories=test_order, ordered=True)

        # Instead of sorting by mean, we sort by the numeric value from shorten_label_into_number
    df = df.sort_values(by="Test", key=lambda col: col.map(shorten_label_into_number))
    return df


def create_mean_df(df):
    df = df.groupby("Test")["Total Query Time"].agg(
        mean="mean", std="std", count="count", max="max", min="min"
    ).reset_index()
    df["stderr"] = df["std"] / np.sqrt(df["count"])
    return df




In [129]:
def load_results_df(path):
    results_df = pd.read_csv(f"{path}")

    # remove Unamed: 0 column
    if "Unnamed: 0" in results_df.columns:
        results_df = results_df.drop(columns=['Unnamed: 0'])

    # Rename Total query time to Total Query Time if it exists
    if "Total query time" in results_df.columns:
        results_df = results_df.rename(columns={"Total query time": "Total Query Time"})

    

    return results_df




# Phase 1

In [ ]:
results_df = load_results_df(RESULTS_PATH_1)
results_mean_df = create_mean_df(results_df)
results_mean_df = prepare_df_for_plot(results_mean_df)
plot_mean_results(results_mean_df)

In [ ]:
# Loop over each load value (0 to 9)
for load in range(10):
    # Filter the DataFrame for the current load and tests in test_order
    df_load = results_df[(results_df["Load"] == load) & (results_df["Test"].isin(test_order))].copy()
    
    # Ensure tests are ordered as defined in test_order
    df_load["Test"] = pd.Categorical(df_load["Test"], categories=test_order, ordered=True)

    # Instead of sorting by mean, we sort by the numeric value from shorten_label_into_number
    df_load = df_load.sort_values(by="Test", key=lambda col: col.map(shorten_label_into_number))
    
    # Create plot for current load
    plot_load_results(df_load, load)


# Phase 3

In [ ]:
results_df = load_results_df(RESULTS_PATH_3)
results_mean_df = create_mean_df(results_df)
results_mean_df = prepare_df_for_plot(results_mean_df)
plot_mean_results(results_mean_df)


In [ ]:
# Loop over each load value (0 to 9)
for load in range(10):
    # Filter the DataFrame for the current load and tests in test_order
    df_load = results_df[(results_df["Load"] == load) & (results_df["Test"].isin(test_order))].copy()
    
    # Ensure tests are ordered as defined in test_order
    df_load["Test"] = pd.Categorical(df_load["Test"], categories=test_order, ordered=True)

    # Instead of sorting by mean, we sort by the numeric value from shorten_label_into_number
    df_load = df_load.sort_values(by="Test", key=lambda col: col.map(shorten_label_into_number))
    
    # Create plot for current load
    plot_load_results(df_load, load)


# Combined

In [134]:
def plot_grouped_comparison(phase1_df, phase3_df, title="Phase 1 vs Phase 3 Comparison", show_values=True):
    """
    Create a grouped bar chart comparing Phase 1 and Phase 3 results.
    Each test has two bars side by side.
    """
    # Merge the dataframes on Test column
    merged_df = pd.merge(phase1_df, phase3_df, on="Test", suffixes=("_phase1", "_phase3"))
    
    # Create shortened labels for x-axis
    short_labels = [shorten_label(test) for test in merged_df["Test"]]
    
    # Define positions and bar width
    x = np.arange(len(merged_df))
    bar_width = 0.35
    
    # Create figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Plot Phase 1 bars
    bars1 = ax.bar(x - bar_width/2, merged_df["mean_phase1"], bar_width, 
                   label="Phase 1", alpha=0.8, capsize=5,
                   yerr=merged_df["stderr_phase1"])
    
    # Plot Phase 3 bars  
    bars2 = ax.bar(x + bar_width/2, merged_df["mean_phase3"], bar_width,
                   label="Phase 3", alpha=0.8, capsize=5,
                   yerr=merged_df["stderr_phase3"])
    
    # Customize the plot
    ax.set_xlabel("Test")
    ax.set_ylabel("Total Query Time (s)")
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(short_labels, rotation=45)
    ax.legend()
    ax.grid(True, axis="y", alpha=0.3)
    
    # Add value labels if requested
    if show_values:
        for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
            height1 = bar1.get_height()
            height2 = bar2.get_height()
            ax.text(bar1.get_x() + bar1.get_width()/2., height1,
                    f'{height1:.1f}', ha='center', va='bottom', fontsize=8)
            ax.text(bar2.get_x() + bar2.get_width()/2., height2,
                    f'{height2:.1f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()


def plot_line_comparison(phase1_df, phase3_df, title="Phase 1 vs Phase 3 Line Comparison", show_values=True):
    """
    Create overlayed line plots comparing Phase 1 and Phase 3 results.
    """
    # Merge the dataframes on Test column
    merged_df = pd.merge(phase1_df, phase3_df, on="Test", suffixes=("_phase1", "_phase3"))
    
    # Create shortened labels for x-axis
    short_labels = [shorten_label(test) for test in merged_df["Test"]]
    
    # Create figure
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Plot Phase 1 line
    ax.errorbar(range(len(merged_df)), merged_df["mean_phase1"], 
                yerr=merged_df["stderr_phase1"], 
                marker='o', linewidth=2, markersize=6, 
                label="Phase 1", capsize=5)
    
    # Plot Phase 3 line
    ax.errorbar(range(len(merged_df)), merged_df["mean_phase3"],
                yerr=merged_df["stderr_phase3"],
                marker='s', linewidth=2, markersize=6,
                label="Phase 3", capsize=5)
    
    # Customize the plot
    ax.set_xlabel("Test")
    ax.set_ylabel("Total Query Time (s)")
    ax.set_title(title)
    ax.set_xticks(range(len(merged_df)))
    ax.set_xticklabels(short_labels, rotation=45)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add value labels if requested
    if show_values:
        for i in range(len(merged_df)):
            ax.text(i, merged_df["mean_phase1"].iloc[i], 
                   f'{merged_df["mean_phase1"].iloc[i]:.1f}', 
                   ha='center', va='bottom', fontsize=8)
            ax.text(i, merged_df["mean_phase3"].iloc[i],
                   f'{merged_df["mean_phase3"].iloc[i]:.1f}',
                   ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()


def plot_side_by_side_comparison(phase1_df, phase3_df, title="Phase 1 vs Phase 3 Side-by-Side", show_values=True):
    """
    Create side-by-side synchronized plots for easy comparison.
    """
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Determine common y-axis limits
    all_means = list(phase1_df["mean"]) + list(phase3_df["mean"])
    all_errors = list(phase1_df["stderr"]) + list(phase3_df["stderr"])
    y_max = max([m + e for m, e in zip(all_means, all_errors)]) * 1.1
    y_min = 0
    
    # Plot Phase 1
    short_labels1 = [shorten_label(test) for test in phase1_df["Test"]]
    x1 = np.arange(len(phase1_df))
    bars1 = ax1.bar(x1, phase1_df["mean"], 
                    color=[colors[test] for test in phase1_df["Test"]],
                    yerr=phase1_df["stderr"], capsize=5, alpha=0.8)
    
    ax1.set_xlabel("Test")
    ax1.set_ylabel("Total Query Time (s)")
    ax1.set_title("Phase 1: Mean Total Query Time")
    ax1.set_xticks(x1)
    ax1.set_xticklabels(short_labels1, rotation=45)
    ax1.set_ylim(y_min, y_max)
    ax1.grid(True, axis="y", alpha=0.3)
    
    # Add value labels for Phase 1 if requested
    if show_values:
        for i, bar in enumerate(bars1):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', fontsize=8)
    
    # Plot Phase 3
    short_labels2 = [shorten_label(test) for test in phase3_df["Test"]]
    x2 = np.arange(len(phase3_df))
    bars2 = ax2.bar(x2, phase3_df["mean"],
                    color=[colors[test] for test in phase3_df["Test"]],
                    yerr=phase3_df["stderr"], capsize=5, alpha=0.8)
    
    ax2.set_xlabel("Test")
    ax2.set_ylabel("Total Query Time (s)")
    ax2.set_title("Phase 3: Mean Total Query Time")
    ax2.set_xticks(x2)
    ax2.set_xticklabels(short_labels2, rotation=45)
    ax2.set_ylim(y_min, y_max)
    ax2.grid(True, axis="y", alpha=0.3)
    
    # Add value labels for Phase 3 if requested
    if show_values:
        for i, bar in enumerate(bars2):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', fontsize=8)
    
    # Add annotations pointing out key differences
    # Find the test with the largest difference
    merged_df = pd.merge(phase1_df, phase3_df, on="Test", suffixes=("_phase1", "_phase3"))
    merged_df["diff"] = abs(merged_df["mean_phase1"] - merged_df["mean_phase3"])
    max_diff_idx = merged_df["diff"].idxmax()
    max_diff_test = merged_df.loc[max_diff_idx, "Test"]
    
    # Add annotation
    fig.suptitle(f"{title}\nLargest difference: {shorten_label(max_diff_test)}", 
                fontsize=16, y=0.98)
    
    plt.tight_layout()
    plt.show()


def plot_vertical_comparison(phase1_df, phase3_df, title="Phase 1 vs Phase 3 Vertical Comparison", show_values=True):
    """
    Create vertically stacked synchronized plots for easy comparison.
    """
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # Determine common y-axis limits
    all_means = list(phase1_df["mean"]) + list(phase3_df["mean"])
    all_errors = list(phase1_df["stderr"]) + list(phase3_df["stderr"])
    y_max = max([m + e for m, e in zip(all_means, all_errors)]) * 1.1
    y_min = 0
    
    # Plot Phase 1
    short_labels1 = [shorten_label(test) for test in phase1_df["Test"]]
    x1 = np.arange(len(phase1_df))
    bars1 = ax1.bar(x1, phase1_df["mean"], 
                    color=[colors[test] for test in phase1_df["Test"]],
                    yerr=phase1_df["stderr"], capsize=5, alpha=0.8)
    
    ax1.set_ylabel("Total Query Time (s)")
    ax1.set_title("Phase 1: Mean Total Query Time")
    ax1.set_xticks(x1)
    ax1.set_xticklabels(short_labels1, rotation=45)
    ax1.set_ylim(y_min, y_max)
    ax1.grid(True, axis="y", alpha=0.3)
    
    # Add value labels for Phase 1 if requested
    if show_values:
        for i, bar in enumerate(bars1):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', fontsize=8)
    
    # Plot Phase 3
    short_labels2 = [shorten_label(test) for test in phase3_df["Test"]]
    x2 = np.arange(len(phase3_df))
    bars2 = ax2.bar(x2, phase3_df["mean"],
                    color=[colors[test] for test in phase3_df["Test"]],
                    yerr=phase3_df["stderr"], capsize=5, alpha=0.8)
    
    ax2.set_xlabel("Test")
    ax2.set_ylabel("Total Query Time (s)")
    ax2.set_title("Phase 3: Mean Total Query Time")
    ax2.set_xticks(x2)
    ax2.set_xticklabels(short_labels2, rotation=45)
    ax2.set_ylim(y_min, y_max)
    ax2.grid(True, axis="y", alpha=0.3)
    
    # Add value labels for Phase 3 if requested
    if show_values:
        for i, bar in enumerate(bars2):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Phase Comparison Section
## Load and prepare data for both phases
phase1_results_df = load_results_df(RESULTS_PATH_1)
phase1_mean_df = create_mean_df(phase1_results_df)
phase1_mean_df = prepare_df_for_plot(phase1_mean_df)

phase3_results_df = load_results_df(RESULTS_PATH_3)
phase3_mean_df = create_mean_df(phase3_results_df)
phase3_mean_df = prepare_df_for_plot(phase3_mean_df)

# Create all comparison plots
plot_grouped_comparison(phase1_mean_df, phase3_mean_df)
plot_line_comparison(phase1_mean_df, phase3_mean_df)
plot_side_by_side_comparison(phase1_mean_df, phase3_mean_df)
plot_vertical_comparison(phase1_mean_df, phase3_mean_df)